<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/SentencePiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install sentencepiece

In [14]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [15]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")
train_df=pd.read_csv('IMDb_Reviews.csv')

In [16]:
train_df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [17]:
print(len(train_df))

50000


In [18]:
#센텐스피스의 입력으로 사용하기 위해서는 데이터프레임을 txt파일로 저장해야 한다.
with open('imdb_review.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(train_df['review']))

In [19]:
#센텐스피스는 전처리나 사전 토큰화 없이 단어 분리 토큰화를 할 수 있다. -> 모든 언어 적용 가능
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')
#input : 학습시킬 파일 -> 위 단락에 만든 imdb_review.txt
#model_prefix : 만들어질 모델 이름 -> imdb
#vocab_size : 단어 집합의 크기 -> 5000
#model_type : 사용할 모델(unigram(default), bpe, char, word) -> bpe
#max_sentence_length : 문장의 최대 길이 -> 9999
#pad_id, pad_piece : pad token id와 값
#unk_id, unk_piece : unknown token id와 값
#bos_id, bos_piece : begin of sentence token id와 값
#eos_id, eos_piece : end of sentence token id와 값
#user_defined_symbols : 사용자 정의 토큰

In [20]:
vocab_list=pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE) #sep으로 구분자 \t, qouting=csv.QUOTE_NONE 으로 둘러쌓인 문자를 신경쓰지 않는다.
vocab_list.sample(10)

,0,1
4754,▁finale,-4751
1184,▁remem,-1181
4834,▁safe,-4831
1058,ident,-1055
4756,▁results,-4753
746,▁laugh,-743
4016,xual,-4013
1714,els,-1711
70,ad,-67
4714,▁confront,-4711


In [21]:
len(vocab_list)

5000

In [22]:
sp=spm.SentencePieceProcessor()
vocab_file='imdb.model'
sp.load(vocab_file)

True

In [25]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 624, 4950, 4926, 139, 170, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91]



In [26]:
#단어집합 크기 확인
sp.GetPieceSize()

5000

In [31]:
#정수로 받아서 맵핑되는 서브 워드로 변환
sp.IdToPiece(40)

'▁l'

In [32]:
#서브워드로 받아서 맵핑되는 정수로 변환
sp.PieceToId('▁l')

40

In [33]:
#정수 시퀀스를 문장으로 변환(decode)
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'Iul wa fall aold timeooland to film'

In [34]:
#서브워드 시퀀스를 문장으로 변환decode
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

In [35]:
#문장을 정수 시퀀스나 서브워드 시퀀스로 변환(encode)
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91]


In [47]:
#네이버 영화 리뷰 토큰화
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [48]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df=pd.read_table('ratings.txt')
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [49]:
print("리뷰 개수 : ", len(naver_df))

리뷰 개수 :  200000


In [50]:
naver_df.isnull().any()

id          False
document     True
label       False
dtype: bool

In [51]:
naver_df=naver_df.dropna(how='any') #nan인 행 제거
len(naver_df)

199992

In [52]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(naver_df['document'])) #모두 문자열로 입력받고 행마다 개행처리

In [53]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')
#센텐스피스를 통해 단어 집합 생성

In [55]:
vocab_list=pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10) #무작위로 10개 출력

,0,1
3296,ᄏ,-3293
923,▁재미가,-920
1643,러운,-1640
3425,명,-3422
2190,▁재밌었어요,-2187
3492,매,-3489
4096,폰,-4093
451,▁ᄏ,-448
4860,렜,-4857
74,토리,-71


In [56]:
len(vocab_list) #vocab에 학습시킨 vocab_size가 5000임

5000

In [57]:
sp=spm.SentencePieceProcessor()
vocab_file='naver.model' #model이름을 위에서 naver로 했었음
sp.load(vocab_file)

True

In [58]:
lines=[
    "뭐 이딴 것도 영화냐.",
    "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[136, 970, 1299, 2593, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 204, 825, 121]



In [59]:
sp.GetPieceSize()

5000

In [60]:
sp.IdToPiece(4)

'영화'

In [75]:
sp.PieceToId('영화')

4

In [76]:
sp.DecodeIds([352, 2085, 4, 187,])

'진짜최고의영화입니다'

In [64]:
sp.DecodePieces(['_진짜', '_최고의', '_영화입니다','_ㅋㅋ'])

'_진짜_최고의_영화입니다_ㅋㅋ'

In [78]:
print(sp.encode('안녕하세요?', out_type=str))
print(sp.encode('안녕하세요?', out_type=int))

['▁안', '녕', '하세요', '?']
[41, 4240, 2437, 3329]
